# Exporting data from BigQuery to Google Cloud Storage

In this notebook, we export BigQuery data to GCS so that we can reuse our Keras model that was developed on CSV data.

In [1]:
from google import api_core
from google.cloud import bigquery

Change the following cell as necessary:

In [2]:
# Change below if necessary
PROJECT = !gcloud config get-value project  # noqa: E999
PROJECT = PROJECT[0]
BUCKET = PROJECT

OUTDIR = "gs://{bucket}/taxifare/data".format(bucket=BUCKET)

%env PROJECT=$PROJECT
%env BUCKET=$BUCKET
%env OUTDIR=$OUTDIR

env: PROJECT=qwiklabs-gcp-00-eeb852ce8ccb
env: BUCKET=qwiklabs-gcp-00-eeb852ce8ccb
env: OUTDIR=gs://qwiklabs-gcp-00-eeb852ce8ccb/taxifare/data


## Create BigQuery tables

If you haven not already created a BigQuery dataset for our data, run the following cell:

In [3]:
bq = bigquery.Client(project=PROJECT)
dataset = bigquery.Dataset(bq.dataset("taxifare"))

try:
    bq.create_dataset(dataset)
    print("Dataset created")
except api_core.exceptions.Conflict:
    print("Dataset already exists")

Dataset created


Let's create a table with 1 million examples.

Note that the order of columns is exactly what was in our CSV files.

In [10]:
%%bigquery
SELECT
    
    COUNT(pickup_datetime)
    
FROM `nyc-tlc.yellow.trips`
WHERE ABS(MOD(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING)), 1000)) = 2

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.29s/rows]


,f0_
0,1118851


In [4]:
%%bigquery

CREATE OR REPLACE TABLE taxifare.feateng_training_data AS

SELECT
    (tolls_amount + fare_amount) AS fare_amount,
    pickup_datetime,
    pickup_longitude AS pickuplon,
    pickup_latitude AS pickuplat,
    dropoff_longitude AS dropofflon,
    dropoff_latitude AS dropofflat,
    passenger_count*1.0 AS passengers,
    'unused' AS key
FROM `nyc-tlc.yellow.trips`
WHERE ABS(MOD(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING)), 1000)) = 1
AND
    trip_distance > 0
    AND fare_amount >= 2.5
    AND pickup_longitude > -78
    AND pickup_longitude < -70
    AND dropoff_longitude > -78
    AND dropoff_longitude < -70
    AND pickup_latitude > 37
    AND pickup_latitude < 45
    AND dropoff_latitude > 37
    AND dropoff_latitude < 45
    AND passenger_count > 0

Query complete after 0.01s: 100%|██████████| 3/3 [00:00<00:00, 1835.05query/s]                        


""


Make the validation dataset be 1/10 the size of the training dataset.

In [5]:
%%bigquery

CREATE OR REPLACE TABLE taxifare.feateng_valid_data AS

SELECT
    (tolls_amount + fare_amount) AS fare_amount,
    pickup_datetime,
    pickup_longitude AS pickuplon,
    pickup_latitude AS pickuplat,
    dropoff_longitude AS dropofflon,
    dropoff_latitude AS dropofflat,
    passenger_count*1.0 AS passengers,
    'unused' AS key
FROM `nyc-tlc.yellow.trips`
WHERE ABS(MOD(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING)), 10000)) = 2
AND
    trip_distance > 0
    AND fare_amount >= 2.5
    AND pickup_longitude > -78
    AND pickup_longitude < -70
    AND dropoff_longitude > -78
    AND dropoff_longitude < -70
    AND pickup_latitude > 37
    AND pickup_latitude < 45
    AND dropoff_latitude > 37
    AND dropoff_latitude < 45
    AND passenger_count > 0

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1530.77query/s]                        


""


## Export the tables as CSV files

In [6]:
%%bash

echo "Deleting current contents of $OUTDIR"
gsutil -m -q rm -rf $OUTDIR

echo "Extracting training data to $OUTDIR"
bq --location=US extract \
   --destination_format CSV  \
   --field_delimiter "," --noprint_header \
   taxifare.feateng_training_data \
   $OUTDIR/taxi-train-*.csv

echo "Extracting validation data to $OUTDIR"
bq --location=US extract \
   --destination_format CSV  \
   --field_delimiter "," --noprint_header \
   taxifare.feateng_valid_data \
   $OUTDIR/taxi-valid-*.csv

gsutil ls -l $OUTDIR

Deleting current contents of gs://qwiklabs-gcp-00-eeb852ce8ccb/taxifare/data
Extracting training data to gs://qwiklabs-gcp-00-eeb852ce8ccb/taxifare/data
Extracting validation data to gs://qwiklabs-gcp-00-eeb852ce8ccb/taxifare/data
  88345235  2021-10-08T18:04:29Z  gs://qwiklabs-gcp-00-eeb852ce8ccb/taxifare/data/taxi-train-000000000000.csv
   8725746  2021-10-08T18:04:36Z  gs://qwiklabs-gcp-00-eeb852ce8ccb/taxifare/data/taxi-valid-000000000000.csv
TOTAL: 2 objects, 97070981 bytes (92.57 MiB)


CommandException: 1 files/objects could not be removed.
Waiting on bqjob_r5d20b19b51e71cad_0000017c61139946_1 ... (23s) Current status: DONE   
Waiting on bqjob_r6750c81c8ec570c8_0000017c6113fd68_1 ... (3s) Current status: DONE   


In [7]:
!gsutil cat gs://$BUCKET/taxifare/data/taxi-train-000000000000.csv | head -2

28.65,2009-06-18 15:42:25 UTC,-73.874484,40.773973,-73.986938,40.759538,2,unused
16.9,2010-11-27 19:43:00 UTC,-73.950205,40.78001,-74.007307,40.743348,2,unused


Copyright 2019 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.